# Dati elezioni: data processing per caricamento in DAF

## Governo italiano - Ministero dell'Interno - Dipartimento per gli Affari Interni e Territoriali

### Dati rilevazione semestrale del corpo elettorale: 

Dati semestrali disponibili dal 1989 ad oggi

http://dait.interno.gov.it/elezioni/open-data/dati-rilevazione-semestrale-del-corpo-elettorale


Al fine di caricare in formato open e in modo ottimizzato in [DAF](https://dataportal-private.daf.teamdigitale.it/) il dati relativi ai corpo elettorale, è necessario uniformare i dataset disponibili semestralmente e inserire via via in append i dataset nel formato nativo aggiungendo il riferimento temporale del dataset che si sta elaborando.

Es formato nome file:
rileseme_30061989_0.csv
rilesame_ddmmYYYY_0.csv

Note: 

- in alcuni file si usa il separatore delle migliaia ".", altre volte no.

- Il file "rileseme_30062017_0.csv" è un file Excel e va letto come tale

- Per l'anno 1991 c'è un unico file

- Per l'anno 1991 i nomi colonna cambiano ma l'ordine viene mantenuto, assegno anche a questo file le colonne standard salvate in header (header=None)

- Per i seguenti file:

 - 1990-12-31
 - 1991-12-31
 - 1993-06-30
 - 1994-06-30
 - 2000-06-30


 
 Il campo "POPOLAZIONE CENSITA" è di un ordine di grandezza in più (!),viene fatta una correzione ad hoc per i file segnalati

In [19]:
import pandas as pd
import os
import glob
from datetime import datetime
from dateutil import parser

In [26]:
header  =[
"ZONA",
"REGIONE",
"PROVINCIA",
"COMUNE",
"DENOMINAZIONE",
"POPOLAZIONE CENSITA",
"TOTALE SEZIONI",
"DI CUI OSPEDALIER",
"ISCRITTI MASCHI",
"ISCRITTI FEMMINE",
"ISCRITTI TOTALI",
"DICIOTTENNI MASCHI",
"DICIOTTENNI FEMMINE",
"DICIOTTENNI TOTALI",
"ELETTORI ALL'ESTERO MASCHI",
"ELETTORI ALL'ESTERO FEMMINE",
"ELETTORI ALL'ESTERO TOTALI"
]

folder_dati = "DatiRilevazioneSemestraleCorpoElettorale"

list_wrog_pop = [
    "30061993",
    "31121990",
    "30062000",
    "31121991",
    "30061994"]

field_type={'ZONA': object,
       'REGIONE': object,
       'PROVINCIA': object,
       'COMUNE': object,
       'TOTALE SEZIONI': float,
       'ISCRITTI MASCHI':float,
       'ISCRITTI FEMMINA':float,
       'ISCRITTI TOTALI':float}

list_ =[]
# folder with csv manually saved from the link above
folder = glob.glob('./'+folder_dati+'/*.csv')
#loop 
for file_ in folder:
    print (file_)
    file_part = file_.split('_') 
    date_file = file_part[1]
        
    df = pd.read_csv(file_, index_col=None, sep=";", names=header, dtype=field_type, skiprows=1, thousands='.')
    
    df["dataRilevazione"] = pd.to_datetime(date_file, format='%d%m%Y')
    #trim su campo COMUNE
    df["DENOMINAZIONE"] = df["DENOMINAZIONE"].str.strip()
    #se è uno dei file con errori in campo popolazione vai a correggere
    if date_file in list_wrog_pop:
        print (date_file)
        df["POPOLAZIONE CENSITA"] = (df["POPOLAZIONE CENSITA"]/10)
    
    list_.append(df)
frame = pd.concat(list_)

frame.to_csv("./output/DatiRilevazioneSemestraleCorpoElettorale.csv", index= False)

./DatiRilevazioneSemestraleCorpoElettorale/rileseme_31121991_0.csv
31121991
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_30062016_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_31122000_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_31121988_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_31121993_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_31122002_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_30062014_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_30062010_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_31122006_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_31121997_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_30061998_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_30062009_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_31122004_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_30062012_0.csv
./DatiRilevazioneSemestraleCorpoElettorale/rileseme_3

In [21]:
len(frame)

461163

In [22]:
frame.columns

Index(['ZONA', 'REGIONE', 'PROVINCIA', 'COMUNE', 'DENOMINAZIONE',
       'POPOLAZIONE CENSITA', 'TOTALE SEZIONI', 'DI CUI OSPEDALIER',
       'ISCRITTI MASCHI', 'ISCRITTI FEMMINE', 'ISCRITTI TOTALI',
       'DICIOTTENNI MASCHI', 'DICIOTTENNI FEMMINE', 'DICIOTTENNI TOTALI',
       'ELETTORI ALL'ESTERO MASCHI', 'ELETTORI ALL'ESTERO FEMMINE',
       'ELETTORI ALL'ESTERO TOTALI', 'dataRilevazione'],
      dtype='object')

#### Check su popolazione comune per cui era di un ordine di grandezza in più 
Il valore del  dataset orginario è quindi stato corretto

es. Acqui Terme

In [23]:
#check Acqui Terme
frame.DENOMINAZIONE= frame.DENOMINAZIONE.str.strip()
frame[(frame.COMUNE=="0010") & (frame.PROVINCIA=='002') & (frame.REGIONE=='01')].sort_values(by=['POPOLAZIONE CENSITA'])


,ZONA,REGIONE,PROVINCIA,COMUNE,DENOMINAZIONE,POPOLAZIONE CENSITA,TOTALE SEZIONI,DI CUI OSPEDALIER,ISCRITTI MASCHI,ISCRITTI FEMMINE,ISCRITTI TOTALI,DICIOTTENNI MASCHI,DICIOTTENNI FEMMINE,DICIOTTENNI TOTALI,ELETTORI ALL'ESTERO MASCHI,ELETTORI ALL'ESTERO FEMMINE,ELETTORI ALL'ESTERO TOTALI,dataRilevazione
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,7956.0,9344.0,17300.0,36.0,29.0,65.0,282.0,236.0,518.0,2008-06-30
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,8039.0,9389.0,17428.0,37.0,22.0,59.0,258.0,261.0,519.0,2007-12-31
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,7846.0,9113.0,16959.0,30.0,38.0,68.0,281.0,289.0,570.0,2011-06-30
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,8113.0,9449.0,17562.0,31.0,34.0,65.0,230.0,221.0,451.0,2004-06-30
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,0.0,8064.0,9429.0,17493.0,45.0,34.0,79.0,243.0,251.0,494.0,2005-12-31
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,7816.0,9105.0,16921.0,35.0,38.0,73.0,292.0,301.0,593.0,2012-12-31
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,7871.0,9151.0,17022.0,35.0,34.0,69.0,287.0,291.0,578.0,2011-12-31
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,8019.0,9392.0,17411.0,33.0,40.0,73.0,258.0,261.0,519.0,2007-06-30
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,8033.0,9402.0,17435.0,41.0,33.0,74.0,249.0,262.0,511.0,2006-06-30
0,1,01,002,0010,ACQUI TERME,19184.0,24.0,1.0,8028.0,9414.0,17442.0,32.0,33.0,65.0,239.0,233.0,472.0,2005-06-30
